In [10]:
import pandas as pd
import numpy as np

In [11]:
## Merge population and covid dataz

covid = pd.read_csv('./Data/covid_19_data.csv')
pop = pd.read_csv('./Data/population_by_country_2020.csv')

In [12]:
covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [43]:
# rename columns
pop = pop.rename(columns={'Country (or dependency)':'Country', 'Population (2020)':'Population_2020'})
pop.head()

,Country,Population_2020,Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1438207241,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1377233523,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,330610570,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,272931713,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,219992900,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %


In [14]:
## Create ISO3 for datasets
import pycountry

import geopandas as gpd # for reading vector-based spatial data format
#shapefile = '/kaggle/input/natural-earth-maps/ne_110m_admin_0_countries.shp'
shapefile = r'C:\Users\TeYan\OneDrive\Work\Kaggle\COVID19\110m_cultural\ne_110m_admin_0_countries.shp'

# Read shapefile using Geopandas
#gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf = gpd.read_file(shapefile)

# Drop row corresponding to 'Antarctica'
gdf = gdf.drop(gdf.index[159])
## Get the ISO 3166-1 alpha-3 Country Codes

# function for getting the iso code through fuzzy search
def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
    except Exception:
        return np.nan
    else:
        return result[0].alpha_3

In [32]:
# replace country names to fit pycountry
covid.loc[covid['Country/Region'] == 'Mainland China', 'Country/Region'] = 'China'
covid.loc[covid['Country/Region'] == 'South Korea', 'Country/Region'] = 'Korea, Republic of'
covid.loc[covid['Country/Region'] == 'Ivory Coast', 'Country/Region'] = "Côte d'Ivoire"
covid.loc[covid['Country/Region'] == 'Republic of Ireland', 'Country/Region'] = 'Ireland'
covid.loc[covid['Country/Region'] == 'occupied Palestinian territory', 'Country/Region'] = 'Palestine, State of'
covid.loc[covid['Country/Region'] == 'Burma', 'Country/Region'] = 'Myanmar'
covid.loc[covid['Country/Region'] == 'East Timor', 'Country/Region'] = 'Timor-Leste'
covid.loc[covid['Country/Region'] == 'St. Martin', 'Country/Region'] = 'Saint Martin (French part)'
covid.loc[covid['Country/Region'] == 'Bahamas, The', 'Country/Region'] = 'Bahamas'
covid.loc[covid['Country/Region'] == 'Congo (Kinshasa)', 'Country/Region'] = 'Congo, The Democratic Republic of the'
covid.loc[covid['Country/Region'] == 'Congo (Brazzaville)', 'Country/Region'] = 'Congo'
covid.loc[covid['Country/Region'] == 'Laos', 'Country/Region'] = "Lao People's Democratic Republic"
# Remove North Ireland which only has 1 day of record, its most likely merged with UK
covid.loc[covid['Country/Region'] == 'North Ireland'] = ''

# create iso mapping for countries in df
iso_map = {country: do_fuzzy_search(country) for country in covid['Country/Region'].unique()}
# apply the mapping to df
covid['iso3'] = covid['Country/Region'].map(iso_map)

#list(pycountry.countries)

In [33]:
covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,iso3
0,1,01/22/2020,Anhui,China,1/22/2020 17:00,1.0,0.0,0.0,CHN
1,2,01/22/2020,Beijing,China,1/22/2020 17:00,14.0,0.0,0.0,CHN
2,3,01/22/2020,Chongqing,China,1/22/2020 17:00,6.0,0.0,0.0,CHN
3,4,01/22/2020,Fujian,China,1/22/2020 17:00,1.0,0.0,0.0,CHN
4,5,01/22/2020,Gansu,China,1/22/2020 17:00,0.0,0.0,0.0,CHN


In [34]:
noiso = covid[covid['iso3'].isna()]['Country/Region'].unique()
noiso

array(['Macau', 'Others', 'North Ireland', "('St. Martin',)",
       'Channel Islands', 'Gambia, The', 'Cape Verde', 'Diamond Princess',
       'West Bank and Gaza', 'MS Zaandam'], dtype=object)

In [53]:
# replace country names to fit pycountry
pop.loc[pop['Country'] == 'Burma', 'Country'] = 'Myanmar'
pop.loc[pop['Country'] == 'South Korea', 'Country'] = 'Korea, Republic of'
pop.loc[pop['Country'] == 'North Korea', 'Country'] = "Korea, Democratic People's Republic of"
pop.loc[pop['Country'] == 'Czech Republic (Czechia)', 'Country'] = 'Czechia'
pop.loc[pop['Country'] == 'Laos', 'Country'] = "Lao People's Democratic Republic"
pop.loc[pop['Country'] == 'St. Vincent & Grenadines', 'Country'] = 'Saint Vincent and the Grenadines'
pop.loc[pop['Country'] == 'U.S. Virgin Islands', 'Country'] = 'Virgin Islands, U.S.'
pop.loc[pop['Country'] == 'Saint Kitts & Nevis', 'Country'] = 'Saint Kitts and Nevis'
pop.loc[pop['Country'] == 'Faeroe Islands', 'Country'] = 'Faroe Islands'
pop.loc[pop['Country'] == 'Wallis & Futuna', 'Country'] = 'Wallis and Futuna'
pop.loc[pop['Country'] == 'Saint Pierre & Miquelon', 'Country'] = 'Saint Pierre and Miquelon'
pop.loc[pop['Country'] == 'DR Congo', 'Country'] = 'Congo, The Democratic Republic of the'

# create iso mapping for countries in df
iso_map = {country: do_fuzzy_search(country) for country in pop['Country'].unique()}
# apply the mapping to df
pop['iso3'] = pop['Country'].map(iso_map)

In [54]:
noiso = pop[pop['iso3'].isna()]['Country'].unique()
noiso

array(['Sao Tome & Principe', 'Channel Islands', 'Caribbean Netherlands'],
      dtype=object)